In [26]:
import paramiko

account_list = {
    '10.2.14.89':'root/n9pmlXmu26GBSXovAO5Y',
    '10.2.14.90':'root/R8KpnPThIjQYNSdZueMf',
    '10.2.14.91':'root/5mv1n18vh3LR5ORwie7q'
}


In [27]:
for index in account_list:
    account = account_list[index].split('/')
    # ip、用户名、密码
    ip = index
    port = 22
    # print(account)
    user = account[0]
    password = account[1]

    print(ip, port, user, password)

    # 创建SSHClient 实例对象
    ssh = paramiko.SSHClient()
    # 调用方法，表示没有存储远程机器的公钥，允许访问
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    # 连接远程机器，地址，端口，用户名密码
    ssh.connect(ip, port, user, password, timeout=10)

    # 输入linux命令
    command = "cd /www/wwwroot ;ls"
    stdin, stdout, stderr = ssh.exec_command(command)
    # 输出命令执行结果
    result = stdout.read()
    ## bytes 转 str
    result = str(result)
    result = result.split('\\n')
    for i in result: 
        print(i)
    print('\n')

10.2.14.89 22 root n9pmlXmu26GBSXovAO5Y
b'anaconda-ks.cfg
ROOT.war
'


10.2.14.90 22 root R8KpnPThIjQYNSdZueMf
b'1
anaconda-ks.cfg
nginx
'


10.2.14.91 22 root 5mv1n18vh3LR5ORwie7q
b'anaconda-ks.cfg
zabbix-3.4.9
zabbix-3.4.9.tar.gz
'




Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)


In [88]:
import win32com.client as win32

word_app = win32.Dispatch("Word.Application")
word_app.Visible = True

# 打开现有的 Word 文档
doc_path = r"C:\Users\EDY\Desktop\巡检记录\link_test\南京城市学院运维巡检记录-20240710 copy.docx"
doc = word_app.Documents.Open(doc_path)


# 只获取第一个表格的内容
if doc.Tables.Count > 0:  # 确保文档至少有一个表格
    first_table = doc.Tables[0]  # 获取第一个表格
    row_index = 2  # 第二行
    
    # 提取第二行的第 1、2、5 和 6 列的单元格内容
    columns_to_extract = [1, 2, 5, 6]
    for col_index in columns_to_extract:
        try:
            cell = first_table.Cell(Row=row_index, Column=col_index)
            text = cell.Range.Text.strip().replace("\r\x07", "")  # 清理掉单元格结束符
            if text:
                print(f"第一个表格的第二行，第 {col_index} 列的单元格内容: {text}")
        except Exception as e:
            print(f"无法访问第一个表格的第二行，第 {col_index} 列的单元格: {e}")





第一个表格的第二行，第1列的单元格内容: 2024/10/18
第一个表格的第二行，第2列的单元格内容: Q3
第一个表格的第二行，第5列的单元格内容: 柳毓琳
第一个表格的第二行，第6列的单元格内容: 15317265303


In [43]:
import os
import win32com.client as win32
from datetime import datetime
import re

def replace_quarter_text_in_doc(doc_path):
    # 获取当前时间和季度
    current_year = datetime.now().year
    current_quarter = (datetime.now().month - 1) // 3 + 1

    # 打开 Word 应用
    word_app = win32.Dispatch("Word.Application")
    word_app.Visible = False  # 设置为True可以看到Word界面

    try:
        # 打开现有的 Word 文档
        doc = word_app.Documents.Open(doc_path)
        # 遍历所有 Shape 对象
        for shape in doc.Shapes:
            # 先检查该 Shape 是否有 TextFrame 属性并且 TextFrame 包含文本
            if hasattr(shape, "TextFrame") and shape.TextFrame.HasText:
                text = shape.TextFrame.TextRange.Text.strip()
                # 使用正则表达式替换符合模式的文本
                new_text = re.sub(r"\d{4}年第.*季度运维巡检", f"{current_year}年第{current_quarter}季度运维巡检", text)
                if new_text != text:
                    shape.TextFrame.TextRange.Text = new_text
                    print(f"{text} 替换为: {new_text}")

        # # 遍历文档中的所有表格，逐行遍历所有单元格
        # for table in doc.Tables:
        #     for row in range(1, table.Rows.Count + 1):
        #         for col in range(1, table.Columns.Count + 1):
        #             try:
        #                 cell_text = table.Cell(row, col).Range.Text.strip()
        #                 if cell_text in ["巡检时间", "季度", "巡检人员", "电话"]:
        #                     table.Cell(row, col).Range.Text = "已替换"
        #             except:
        #                 # 忽略因合并单元格导致的访问错误
        #                 continue

        # 保存并关闭文档
        doc.Save()
    except Exception as e:
        print(f"打开或处理文档时出错: {e}")
    finally:
        # 确保关闭文档和 Word 应用
        if 'doc' in locals():  # 确保 doc 已成功打开
            doc.Close()
        word_app.Quit()

# 调用函数，传入文档路径
replace_quarter_text_in_doc(r"C:\Users\EDY\Desktop\巡检记录\link_test\南京城市学院运维巡检记录-20240710 copy.docx")


南京城市学院2020年第撒季度运维巡检统计报告 替换为: 南京城市学院2024年第4季度运维巡检统计报告
